## Purpose

The purpose of this code is to acquire data on article quality predictions from ORES. 

The article source data comes from English Wikipedia, the text of which is licensed under "Creative Commons Attribution Share-Alike license" (https://en.wikipedia.org/wiki/Wikipedia:Text_of_the_Creative_Commons_Attribution-ShareAlike_3.0_Unported_License)

We will be using the MediaWiki REST API for English Wikipedia. To get more information on the API please use the following link: https://www.mediawiki.org/wiki/API:Main_page. The following link may also be helpful when looking for more documentation: https://www.mediawiki.org/wiki/API:Info

We will leverage code developed by Dr. David W. McDonald for use in Data 512  which is provided under Creative Commons CC-BY license. (https://creativecommons.org/ and https://creativecommons.org/licenses/by/4.0/). The file can be found at this link: https://drive.google.com/drive/folders/1FtvWV31DHE8HIMdEsPGuCXPz0PMvShfl

We will also be using the ORES API. Information on the API itself can be found at https://www.mediawiki.org/wiki/ORES, with original API documentation from https://ores.wikimedia.org/docs and new LiftWing documentation from https://wikitech.wikimedia.org/wiki/Machine_Learning/LiftWing/Usage.

We will begin by reading in standard Python libraries.

In [1]:
#Import python libraries
import json
import time
import urllib.parse
import requests
import pandas as pd

Next we will read in the us_cities_by_state_SEPT.2023.csv file from raw_data to create a list of articles which we want to feed in to ORES

In [16]:
#Read in info using pandas
cities_by_st_df = pd.read_csv('../raw_data/us_cities_by_state_SEPT.2023.csv')
cities_by_st_df.head()

#Get a list of page_titles
page_titles = list(cities_by_st_df['page_title'])

#Check if all the page titles were captures
if len(page_titles) == len(cities_by_st_df):
    print("All {0} page titles captured.".format(len(page_titles)))
else:
    print("Not all page titles captured")
    
#Check if page titles are unique
if len(page_titles) == len(cities_by_st_df['page_title'].unique()):
    print("All {0} page titles are unique.".format(len(page_titles)))
else:
    print("WARNING: Page titles are not unique. "+
          "There are {0} page titles and {1} unique titles - ".format(
              len(page_titles), len(cities_by_st_df['page_title'].unique()))
          + "a delta of {0}.".format(len(page_titles)-
                                len(cities_by_st_df['page_title'].unique())))

All 22157 page titles captured.


The user may notice a warning that all page titles are not unique. We will remove duplicated rows in the scraped file us_cities_by_state_SEPT.2023.csv to reduce later processing.

In [45]:
#Removing dupes
cities_dedupe = cities_by_st_df.drop_duplicates()

#Rebuilding the page titles list
page_titles_dedupe = list(cities_dedupe['page_title'])

#Rechecking if page titles are unique
if len(page_titles_dedupe) == len(cities_dedupe['page_title'].unique()):
    print("All {0} page titles are unique.".format(len(page_titles_dedupe)))
else:
    print("WARNING: Page titles are not unique. "+
          "There are {0} page titles and {1} unique titles - ".format(
              len(page_titles_dedupe),
              len(cities_dedupe['page_title'].unique()))
          + "a delta of {0}.".format(len(page_titles_dedupe)-
                                len(cities_dedupe['page_title'].unique())))

It is possible that page titles are still not unique. We will visually inspect these page titles to see if there is any way to reduce later processing.

In [46]:
#Getting list of non-unique city names
#The following code was taken from Stack Overflow (https://stackoverflow.com/questions/9835762/how-do-i-find-the-duplicates-in-a-list-and-create-another-list-with-them)
import collections
dupe_titles = [item for item, count in collections.Counter(page_titles_dedupe).items() if count > 1]
print(dupe_titles)

['2020 United States census', '2010 United States census', 'County (United States)', 'Population']


Using the files originally obtained for this analysis we will find that there are "cities" listed in article titles which are not actually cities at all (e.g., "Population" or "2020 United States census"). We will remove those non-cities here given that the last n characters of a citie's name should be equal to the value in the first column of the us_cities_by_state_SEPT.2023.csv.

In [80]:
#Removing non-cities from the deduped city file
cities_final_df = cities_dedupe.loc[~cities_dedupe['page_title'].isin(dupe_titles)]

In [67]:
'''
#This section has a more robust look at the titles, but there appears
to be multiple naming conventions so we'll settle for the easy removals above

#making new col w/ last n char from cities_dedupe
#cities_dedupe['title_state'] = cities_dedupe['page_title'][:-len(cities_dedupe['state'])]

#cities_dedupe.insert(len(cities_dedupe.columns), 'len', len(cities_dedupe['state']))
cities_dedupe['title_state'] = None
for index, row in cities_dedupe.iterrows():
    row['title_state'] = row['page_title'][-len(row['state']):]
cities_dedupe
cities_dedupe.loc[cities_dedupe['state'] != cities_dedupe['title_state']]'''

"\n#This section has a more robust look at the titles, but there appears\nto be multiple naming conventions so we'll settle for the easy removals above\n\n#making new col w/ last n char from cities_dedupe\n#cities_dedupe['title_state'] = cities_dedupe['page_title'][:-len(cities_dedupe['state'])]\n\n#cities_dedupe.insert(len(cities_dedupe.columns), 'len', len(cities_dedupe['state']))\ncities_dedupe['title_state'] = None\nfor index, row in cities_dedupe.iterrows():\n    row['title_state'] = row['page_title'][-len(row['state']):]\ncities_dedupe\ncities_dedupe.loc[cities_dedupe['state'] != cities_dedupe['title_state']]"

Now we will access the page info from the MediaWiki REST API for English Wikipedia articles. This code was taken from the file provided by Professor McDonald. We will begin by copying the code which sets up the constants for the API call. Some changes were made to keep comments to 1 line and to alter the name of the example list of articles.

In [82]:
# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - 
#we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique 
#to the person making the request
REQUEST_HEADERS = {
    'User-Agent': '<ekrolen@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# Example article list
ex_article_titles = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This is a string of additional page properties that can be returned see 
#the Info documentation for what can be included. If you don't want any 
#this can simply be the empty string
#PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # simplify - single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

Next we copy the API request procedure developed by Professor McDonald. Per his comments, "The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title."

In [ ]:
#Page info requester
def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

#The following code can be run to verify the API is pulling correctly
'''print(f"Getting page info data for: {ex_article_titles[3]}")
info = request_pageinfo_per_article(ex_article_titles[3])
print(json.dumps(info,indent=4))'''

Next we will create a dictionary which contains each of the article titles as keys and their revision ids (lastrevid from request_pageinfo_per_article) for input into the ORES API

In [108]:
#Creating the final list of titles to pull
article_titles = list(cities_final_df['page_title'])

#Pulling lastrevid for all of the articles & saving in a dictionary
page_info_dict = {}
for i in range(len(article_titles)):
    info = request_pageinfo_per_article(article_titles[i])
    page_info_dict[article_titles[i]] = info['query']['pages'][list(info['query']['pages'].keys())[0]]['lastrevid']
    if i%10 == 0:
        print(str(i)+" of "+str(len(article_titles))+" have been written")

0 of 21515 have been written
10 of 21515 have been written
20 of 21515 have been written
30 of 21515 have been written
40 of 21515 have been written
50 of 21515 have been written
60 of 21515 have been written
70 of 21515 have been written
80 of 21515 have been written
90 of 21515 have been written
100 of 21515 have been written
110 of 21515 have been written
120 of 21515 have been written
130 of 21515 have been written
140 of 21515 have been written
150 of 21515 have been written
160 of 21515 have been written
170 of 21515 have been written
180 of 21515 have been written
190 of 21515 have been written
200 of 21515 have been written
210 of 21515 have been written
220 of 21515 have been written
230 of 21515 have been written
240 of 21515 have been written
250 of 21515 have been written
260 of 21515 have been written
270 of 21515 have been written
280 of 21515 have been written
290 of 21515 have been written
300 of 21515 have been written
310 of 21515 have been written
320 of 21515 have b

2600 of 21515 have been written
2610 of 21515 have been written
2620 of 21515 have been written
2630 of 21515 have been written
2640 of 21515 have been written
2650 of 21515 have been written
2660 of 21515 have been written
2670 of 21515 have been written
2680 of 21515 have been written
2690 of 21515 have been written
2700 of 21515 have been written
2710 of 21515 have been written
2720 of 21515 have been written
2730 of 21515 have been written
2740 of 21515 have been written
2750 of 21515 have been written
2760 of 21515 have been written
2770 of 21515 have been written
2780 of 21515 have been written
2790 of 21515 have been written
2800 of 21515 have been written
2810 of 21515 have been written
2820 of 21515 have been written
2830 of 21515 have been written
2840 of 21515 have been written
2850 of 21515 have been written
2860 of 21515 have been written
2870 of 21515 have been written
2880 of 21515 have been written
2890 of 21515 have been written
2900 of 21515 have been written
2910 of 

TypeError: 'NoneType' object is not subscriptable

In [114]:
little_list = article_titles[4370:4390]
print(little_list)
'''little_page_info_dict = {}
for i in range(len(little_list)):
    info = request_pageinfo_per_article(little_list[i])
    little_page_info_dict[article_titles[i]] = info['query']['pages'][list(info['query']['pages'].keys())[0]]['lastrevid']
    print(info)'''

['St. Peter, Illinois', 'St. Rose, Illinois', 'Salem, Illinois', 'Sammons Point, Illinois', 'Sandoval, Illinois', 'Sandwich, Illinois', 'San Jose, Illinois', 'Sauget, Illinois', 'Sauk Village, Illinois', 'Saunemin, Illinois', 'Savanna, Illinois', 'Savoy, Illinois', 'Sawyerville, Illinois', 'Saybrook, Illinois', 'Scales Mound, Illinois', 'Schaumburg, Illinois', 'Schiller Park, Illinois', 'Schram City, Illinois', 'Sciota, Illinois', 'Scottville, Illinois']


"little_page_info_dict = {}\nfor i in range(len(little_list)):\n    info = request_pageinfo_per_article(little_list[i])\n    little_page_info_dict[article_titles[i]] = info['query']['pages'][list(info['query']['pages'].keys())[0]]['lastrevid']\n    print(info)"